In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("afp.csv")

In [44]:
import re
import datetime
def jadate_to_datetime(jadate):
    t = map(int, re.sub(r"(年|月|日|:)"," ", jadate).split())
    return pd.Timestamp(datetime.datetime(*t, 0))
    
jadate_to_datetime(df['day'][0])

Timestamp('2018-07-03 07:06:00')

In [18]:
df.shape, df.dropna().shape

((69294, 4), (69213, 4))

In [19]:
df = df.dropna()

In [45]:
df['day_fixed'] = list(map(jadate_to_datetime, df['day'].tolist()))

In [46]:
df.head()

,body,day,title,url,day_fixed
0,内戦が続くシリアの南部で、ロシアの支援を受けた政府軍が進める軍事作戦により、27万人以上が避...,2018年7月3日 7:06,シリア南部の避難民27万人超える 政府軍攻勢で急増 国連発表,http://www.afpbb.com/articles/-/3180932,2018-07-03 07:06:00
1,ドナルド・トランプ（Donald Trump）米大統領とメキシコの次期大統領に決まったアンド...,2018年7月3日 10:55,メキシコ次期大統領、トランプ氏と初の電話会談 「素晴らしい」滑り出し,http://www.afpbb.com/articles/-/3180945,2018-07-03 10:55:00
2,中国の電気自動車メーカー、比亜迪集団（BYD）が青海省（Qinghai）に建設した車載電池工...,2018年7月3日 9:22,比亜迪の電池工場、青海で操業開始 世界最大規模,http://www.afpbb.com/articles/-/3180880,2018-07-03 09:22:00
3,ベルギーとフランス、ドイツ3か国の当局は、フランスの首都パリ郊外で先月30日に開かれたイラン...,2018年7月3日 9:03,イラン反体制派の集会に攻撃計画か 欧州で外交官ら6人拘束,http://www.afpbb.com/articles/-/3180931,2018-07-03 09:03:00
4,スペインのNGO「プロアクティバ・オープン・アームズ（Proactiva Open Arms...,2018年7月3日 12:13,動画：地中海で移民59人救助、スペインへ イタリアとマルタは入港拒否,http://www.afpbb.com/articles/-/3180976,2018-07-03 12:13:00


In [148]:
import numpy as np
from functools import partial
from dateutil import parser

def diff_date(t1, t2):
    if t1 < t2:
        tmp = t1
        t1 = t2
        t2 = tmp
    return (t1-t2).to_timedelta64().astype("timedelta64[h]")/np.timedelta64(1,'h')

def within(t1, base, hour=12):
    return diff_date(t1, base) <= hour

def get_around_12(df, base, hour=12, column="day_fixed"):
    func = partial(within, base=base, hour=hour)
    return df[list(map(func, df[column].tolist()))]

In [158]:
def get_day(start, plus):
    return pd.Timestamp(start + datetime.timedelta(days=plus))


start = datetime.datetime(2017,2,1,12,0,0)
day = get_day(start, 0)
targets = get_around_12(df, day, hour=12*30)

In [155]:
import MeCab
def extract_data(d, tagger, column1, column2):
    return tagger.parse(d[1][column1] + " " + d[1][column2].split("。")[0])

tagger = MeCab.Tagger("-Owakati")
func = partial(extract_data, tagger=tagger, column1="title", column2="body")

df['data'] = list(map(func, df.iterrows()))

In [157]:
df.to_csv("progress_afp.csv", index=False)

In [193]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import itertools
stps = stopwords.words("japanese")
stps = stps + [''.join(list(stp)) for stp in list(itertools.product(stps, stps))]
count_vectors = CountVectorizer(stop_words=stps).fit_transform(targets['data']).toarray()

In [194]:
from sklearn.metrics.pairwise import cosine_similarity
similarities = cosine_similarity(count_vectors)

In [195]:
import markov_clustering as mc
result = mc.run_mcl(similarities)
clusters = mc.get_clusters(result)

In [202]:
def diff_date(t1, t2):
    if t1 < t2:
        tmp = t1
        t1 = t2
        t2 = tmp
    return (t1-t2).to_timedelta64().astype("timedelta64[h]")/np.timedelta64(1,'h')

def exp_diff(t1, t2):
    return np.exp(diff_date(t1, t2)/24)

def matrix_diffdate(days):
    results = []
    for day in days:
        func = partial(diff_date, t2=day)
        results.append(list(map(func, days)))
    return results

In [203]:
days_matrix = matrix_diffdate(targets['day_fixed'])

In [205]:
days_matrix = np.array(days_matrix)

In [207]:
days_matrix.shape, similarities.shape

((580, 580), (580, 580))

In [219]:
for i, cluster in enumerate(clusters):
    if len(cluster) > 1:
        print("[[[cluster {}]]]".format(i))
        for i, c in enumerate(cluster[:3]):
            print("{}) :{}".format(i, targets.iloc[c]['data']))
        print("")

[[[cluster 0]]]
0) :ロシア は 「 大統領 選 に 干渉 する な 」 仏 外相 が 警告 フランス の ジャンマルク・エロー （ Jean - Marc Ayrault ） 外相 は 15 日 、 4 ～ 5 月 に 行わ れる 大統領 選 を めぐり 、 ロシア に対して 昨年 の 米 大統領 選 と 同様 の 干渉 を 行わ ない よう 強く 警告 し た 

1) :米 労働 長官 候補 、 指名 を 辞退 トランプ 政権 に 新た な 打撃 （ 更新 ） ドナルド ・ トランプ （ Donald Trump ） 米 大統領 が 労働 長官 に 指名 し て い た 米 大手 ファストフード チェーン 経営 者 の アンドリュー・パズダー （ Andrew Puzder ） 氏 （ 66 ） が 15 日 、 指名 を 辞退 し た 

2) :中東 和平 「 2 国家 共存 」 政策 と 決別 か 、 米 ホワイトハウス 高官 が 示唆 米 ホワイトハウス （ White House ） の 高官 は 14 日 、 同国 が 中東 和平 の 実現 に 向け て 数 十 年 にわたり 支持 し て き た イスラエル と パレスチナ の 「 2 国家 共存 」 から の 決別 を 示唆 し た 


[[[cluster 1]]]
0) :NZ クライストチャーチ で 森林 火災 、 非常 事態 を 宣言 ニュージーランド 南島 の クライストチャーチ （ Christchurch ） で 大 規模 な 森林 火災 が 発生 し 、 地元 当局 に よる と 16 日 まで に 家屋 少なくとも 11 棟 が 被災 、 約 1000 人 が 避難 を 余儀なく さ れ て いる 

1) :恵み の 雨 一転 、 深刻 な 暴風雨 被害 に 非常 事態 宣言 米 カリフォルニア 昨年 12 月 から 今月 にかけて 米 カリフォルニア （ California ） 州 を 襲っ た 暴風雨 による 被害 に 対応 する ため 、 ジェリー ・ ブラウン （ Jerry Brown ） 同 州 知事 は 23 日 、 非常 事態 を 宣言 し た 


[[[cluster 2]]]
0) :動画 配信 中 に 銃撃 、 幼児 ら 2 人 

In [212]:
sim2days = np.divide(similarities, days_matrix)
sim2days = np.nan_to_num(sim2days)
result2 = mc.run_mcl(sim2days)
clusters2 = mc.get_clusters(result2)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: RuntimeWarning: invalid value encountered in true_divide
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:32: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims)


In [218]:
for i, cluster in enumerate(clusters2):
    if len(cluster) > 1:
        print("[[[cluster {}]]]".format(i))
        for i, c in enumerate(cluster[:3]):
            print("{}) :{}".format(i, targets.iloc[c]['data']))
        print("")

[[[cluster 79]]]
0) :フィリピン で M 6 . 5 の 地震 、 4 人 死亡 （ 更新 、 写真 追加 ） フィリピン 南部 ミンダナオ （ Mindanao ） 島 の スリガオ （ Surigao ） で 10 日 夜 、 マグニチュード （ M ） 6 . 5 の 強い 地震 が あり 、 一夜 明け これ まで に 少なくとも 4 人 が 死亡 し た 

1) :台湾 南部 で M 5 . 6 の 地震 、 4 人 負傷 米 地質 調査 所 （ USGS ） に よる と 、 台湾 南部 で 11 日 未明 、 マグニチュード （ M ） 5 . 6 の 地震 が 起き た 


[[[cluster 178]]]
0) :チリ 裁判所 、 容疑 者 拘束 認め ず 仏 の 邦人 留学生 不明 事件 （ 更新 ） 仏 ブザンソン （ Besancon ） で 留学 中 だっ た 日本人 学生 の 黒崎 愛 海 （ Narumi Kurosaki ） さん （ 21 ） が 行方 不明 に なっ て いる 事件 で 、 チリ の 裁判所 は 3 日 、 殺人 容疑 で 国際 手配 さ れ て いる 男 の 拘束 を 求め た 仏 当局 の 要請 を 退ける 判断 を 下し た 

1) :米国 、 6 万 人 の ビザ 取り消し トランプ 氏 の 入国 禁止 令 で 米 国務省 は 3 日 、 イスラム 圏 7 か国 出身 者 の 入国 を 禁じ た ドナルド ・ トランプ （ Donald Trump ） 大統領 の 命令 を 受け 、 6 万 人 弱 の ビザ （ 査証 ） を 無効 化 し た と 発表 し た 


[[[cluster 180]]]
0) :スウェーデン 副 首相 が 「 全員 女性 写真 」 トランプ 大統領 に 挑戦 ？ スウェーデン の イサベラ・ロウィン （ Isabella Lovin ） 副 首相 は 3 日 、 ドナルド ・ トランプ （ Donald Trump ） 米 大統領 の 写真 を まね て 、 女性 の 同僚 たち に 囲ま れ て 気候 関連 法案 に 署名 する 自身 の 写真 を 公開 し た 

1) :トランプ 大統領 、 ルーブル美術館 襲撃 受け 「 米国 は 賢く 

In [217]:
clusters2

[(0,),
 (1,),
 (2,),
 (3,),
 (4,),
 (5,),
 (6,),
 (7,),
 (8,),
 (9,),
 (10,),
 (11,),
 (12,),
 (14,),
 (16,),
 (17,),
 (18,),
 (19,),
 (21,),
 (22,),
 (23,),
 (24,),
 (25,),
 (26,),
 (27,),
 (28,),
 (29,),
 (30,),
 (31,),
 (32,),
 (33,),
 (34,),
 (35,),
 (36,),
 (37,),
 (38,),
 (39,),
 (40,),
 (41,),
 (42,),
 (43,),
 (44,),
 (45,),
 (46,),
 (47,),
 (48,),
 (49,),
 (50,),
 (51,),
 (52,),
 (53,),
 (54,),
 (55,),
 (56,),
 (57,),
 (58,),
 (59,),
 (62,),
 (64,),
 (65,),
 (66,),
 (67,),
 (68,),
 (70,),
 (72,),
 (73,),
 (74,),
 (75,),
 (77,),
 (78,),
 (79,),
 (80,),
 (81,),
 (82,),
 (83,),
 (84,),
 (85,),
 (86,),
 (87,),
 (88, 92),
 (90,),
 (91,),
 (93,),
 (96,),
 (97,),
 (102,),
 (105,),
 (109,),
 (110,),
 (112,),
 (113,),
 (114,),
 (115,),
 (116,),
 (117,),
 (118,),
 (119,),
 (120,),
 (121,),
 (122,),
 (123,),
 (124,),
 (125,),
 (126,),
 (127,),
 (128,),
 (129,),
 (130,),
 (131,),
 (134,),
 (136,),
 (139,),
 (140,),
 (141,),
 (144,),
 (146,),
 (147,),
 (148,),
 (149,),
 (150,),
 (151,),
 (1

In [188]:
from nltk.corpus import stopwords

stopwords.words("japanese")

['あそこ',
 'あっ',
 'あの',
 'あのかた',
 'あの人',
 'あり',
 'あります',
 'ある',
 'あれ',
 'い',
 'いう',
 'います',
 'いる',
 'う',
 'うち',
 'え',
 'お',
 'および',
 'おり',
 'おります',
 'か',
 'かつて',
 'から',
 'が',
 'き',
 'ここ',
 'こちら',
 'こと',
 'この',
 'これ',
 'これら',
 'さ',
 'さらに',
 'し',
 'しかし',
 'する',
 'ず',
 'せ',
 'せる',
 'そこ',
 'そして',
 'その',
 'その他',
 'その後',
 'それ',
 'それぞれ',
 'それで',
 'た',
 'ただし',
 'たち',
 'ため',
 'たり',
 'だ',
 'だっ',
 'だれ',
 'つ',
 'て',
 'で',
 'でき',
 'できる',
 'です',
 'では',
 'でも',
 'と',
 'という',
 'といった',
 'とき',
 'ところ',
 'として',
 'とともに',
 'とも',
 'と共に',
 'どこ',
 'どの',
 'な',
 'ない',
 'なお',
 'なかっ',
 'ながら',
 'なく',
 'なっ',
 'など',
 'なに',
 'なら',
 'なり',
 'なる',
 'なん',
 'に',
 'において',
 'における',
 'について',
 'にて',
 'によって',
 'により',
 'による',
 'に対して',
 'に対する',
 'に関する',
 'の',
 'ので',
 'のみ',
 'は',
 'ば',
 'へ',
 'ほか',
 'ほとんど',
 'ほど',
 'ます',
 'また',
 'または',
 'まで',
 'も',
 'もの',
 'ものの',
 'や',
 'よう',
 'より',
 'ら',
 'られ',
 'られる',
 'れ',
 'れる',
 'を',
 'ん',
 '何',
 '及び',
 '彼',
 '彼女',
 '我々',
 '特に',
 '私',
 '私達',
 '貴方',
 '貴方方',
 '日',
 '人',
 '者',
 '月',
 '年',
 '1',
 '2

In [208]:
np.divide

array([[0.5       , 0.75      ],
       [0.83333333, 0.875     ]])